<div class="alert alert-block alert-success">
<b># 🚀 Execução de Circuitos na AWS Braket  

Este notebook permite **executar circuitos quânticos** na **AWS Braket**, com controle de **parâmetros e custos**.  

---

## 🔹 Configuração de Perfis  
Os seguintes **perfis** de execução estão disponíveis:  
- **CONECTIVIDADE** → Teste rápido com poucos shots.  
- **TESTE** → Simulação intermediária.  
- **EXECUTION** → Execução completa (**com alerta de custo se `shots > 100`**).  

Cada perfil carrega **parâmetros de configuração** de um arquivo `.json`.  

**Exemplo de Seleção:**  
```python
perfil = ConfigProfile.EXECUTION
```

---

## 🔹 Seleção de Backend  
Os **dispositivos disponíveis na AWS Braket** são carregados dinamicamente.  
Eles são armazenados no `Enum Backend`.  

**Exemplo de Seleção de Backend:**  
```python
backend = Backend.ARIA_1 
```

---

## 🔹 Estimativa de Custo 💰  
- **Apenas para EXECUTION e shots > 100.**  
- O custo é calculado com base na **tabela oficial da AWS** (**task + preço por shot**).  
- O usuário precisa **confirmar** antes da execução.  

Se `shots <= 100`, o código **executa direto sem alerta**.  

---

## 🔹 Modos de Execução  

### **✔ Síncrono (`run_circuit_sync`)**
- Aguarda o resultado **antes de prosseguir**.  
- Exibe **estatísticas e custo real após a execução**.  

**Exemplo:**  
```python
resultados = run_circuit_sync(perfil, backend, circuit)
print("Resultados:", resultados)
```

### **✔ Assíncrono (`run_circuit_async`)**
- Retorna um **ARN da task** imediatamente.  
- Permite **monitoramento posterior**.  

**Exemplo:**  
```python
task_arn = run_circuit_async(perfil, backend, circuit)
print("Task ARN:", task_arn)
```

---

## 🔹 Monitoramento de Tasks  
O status da execução pode ser verificado manualmente com:  
```python
check_task_status(task_arn)
```

Ou acompanhar automaticamente em um **loop**, esperando a finalização:  
```python
wait_for_task_completion(task_arn)
```

---

## **🔹 Fluxo Completo**
1️⃣ **Escolher perfil e backend:**  
```python
perfil = ConfigProfile.EXECUTION
backend = Backend.AQUILA
circuit = create_circuit()
```

2️⃣ **Executar circuito (Síncrono ou Assíncrono):**  
```python
task_arn = run_circuit_async(perfil, backend, circuit)  # Assíncrono
```

3️⃣ **Acompanhar a execução:**  
```python
wait_for_task_completion(task_arn)
```

---

✅ **Agora você pode rodar circuitos quânticos de forma segura, eficiente e escalável na AWS Braket!** 🚀🔥  
 </b>
</div>

# Instructions if you are running outside AWS cloud

1. Install aws-cli:

`sudo apt install aws-cli`

2. Setup aws credentials:

`aws configure --profile "default"`

3. Create virtual environment 

`python -m venv /path/to/new/virtual/environment`

4. Activate virtual environment

`source /path/to/new/virtual/environment/bin/activate`

### Instalando e Atualizando Dependencias

In [1]:
# Install this modules on your virtual environment: an open source library for interacting
# with quantum computing devices and simulators on Amazon Braket
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Para atualizar as dependencias, caso tenha sido incluidas novas bibliotecas no projeto
!pip freeze > requirements.txt

### Importando classes de inicialização de configuração, devices, pricing e execution

In [3]:
%run devices/aws_backends.ipynb

Dispositivos disponíveis na AWS Braket:
0: Nome: Ankaa-3, Tipo: QPU, ARN: arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3
1: Nome: Aquila, Tipo: QPU, ARN: arn:aws:braket:us-east-1::device/qpu/quera/Aquila
2: Nome: Aria 1, Tipo: QPU, ARN: arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1
3: Nome: Aria 2, Tipo: QPU, ARN: arn:aws:braket:us-east-1::device/qpu/ionq/Aria-2
4: Nome: Forte 1, Tipo: QPU, ARN: arn:aws:braket:us-east-1::device/qpu/ionq/Forte-1
5: Nome: Forte Enterprise 1, Tipo: QPU, ARN: arn:aws:braket:us-east-1::device/qpu/ionq/Forte-Enterprise-1
6: Nome: Garnet, Tipo: QPU, ARN: arn:aws:braket:eu-north-1::device/qpu/iqm/Garnet
7: Nome: SV1, Tipo: SIMULATOR, ARN: arn:aws:braket:::device/quantum-simulator/amazon/sv1
8: Nome: TN1, Tipo: SIMULATOR, ARN: arn:aws:braket:::device/quantum-simulator/amazon/tn1
9: Nome: dm1, Tipo: SIMULATOR, ARN: arn:aws:braket:::device/quantum-simulator/amazon/dm1
Backends ativos: ['ANKAA_3', 'AQUILA', 'FORTE_1', 'FORTE_ENTERPRISE_1', 'GARNET', 'SV1'

In [5]:
%run config/load_configuration.ipynb

In [6]:
%run pricing/Pricing.ipynb


📋 Tabela de Preços das QPUs:
Família de QPU | Preço por Tarefa (USD) | Preço por Shot (USD)
------------------------------------------------------------
Forte      | $0.30000              | $0.08000             
Aria       | $0.30000              | $0.03000             
Garnet     | $0.30000              | $0.00145             
Aquila     | $0.30000              | $0.01000             
Ankaa      | $0.30000              | $0.00090             
------------------------------------------------------------


In [7]:
%run execution/Execution.ipynb

### Criando o Circuito Quântico

In [12]:
from braket.circuits import Circuit

def create_circuit():
    """Cria um circuito de Bell e retorna o objeto Circuit."""
    circuit = Circuit().h(0).h(1).h(2).measure(0).measure(1).measure(2)
    return circuit

### Escolhendo o Backend 

In [9]:
# Listar backends ativos
print("Backends ativos:", list(Backend.__members__.keys()))

Backends ativos: ['ANKAA_3', 'AQUILA', 'FORTE_1', 'FORTE_ENTERPRISE_1', 'GARNET', 'SV1', 'TN1', 'DM1']


### Definição dos parâmetros perfil, backend e circuito 

In [17]:
# Escolha o perfil desejado
perfil = ConfigProfile.TESTE  # Pode ser: ConfigProfile.CONECTIVIDADE, ConfigProfile.TESTE, ConfigProfile.EXECUTION

# Escolher um backend específico
backend = Backend.FORTE_1  # Por exemplo, selecionar o simulador SV1

# Criar um circuito
circuit = create_circuit()

### Testando o código de forma síncrona (fila livre)

In [ ]:
# Executar o circuito com o perfil e backend selecionados
run_circuit_sync(perfil, backend, circuit)

Executando (SÍNCRONO) com 100 shots no perfil 'perfil_2-teste' no backend 'FORTE_1'


### Testando o código de forma assíncrona (fila ocupada)

In [13]:
# Executar o circuito com o perfil e backend selecionados mas de forma assincrona
task_arn = run_circuit_async(perfil, backend, circuit)
print("Task ARN recebida:", task_arn)

Executando (ASSÍNCRONO) com 1 shots no perfil 'perfil_1-conectividade' no backend 'SV1'
📌 Task ARN: arn:aws:braket:us-east-1:709381776175:quantum-task/6f214b68-ae02-4e0a-90b5-487d343b3a52
Task ARN recebida: arn:aws:braket:us-east-1:709381776175:quantum-task/6f214b68-ae02-4e0a-90b5-487d343b3a52


In [16]:
task_arn = "arn:aws:braket:us-west-1:709381776175:quantum-task/2214ee32-8f5b-4db0-a6c9-c5133272b749"
wait_for_task_completion(task_arn)

🕒 Aguardando conclusão da Task arn:aws:braket:us-west-1:709381776175:quantum-task/2214ee32-8f5b-4db0-a6c9-c5133272b749...
🔄 Status da Task arn:aws:braket:us-west-1:709381776175:quantum-task/2214ee32-8f5b-4db0-a6c9-c5133272b749: QUEUED
⌛ Task arn:aws:braket:us-west-1:709381776175:quantum-task/2214ee32-8f5b-4db0-a6c9-c5133272b749 ainda em execução...
🔄 Status da Task arn:aws:braket:us-west-1:709381776175:quantum-task/2214ee32-8f5b-4db0-a6c9-c5133272b749: QUEUED
⌛ Task arn:aws:braket:us-west-1:709381776175:quantum-task/2214ee32-8f5b-4db0-a6c9-c5133272b749 ainda em execução...
🔄 Status da Task arn:aws:braket:us-west-1:709381776175:quantum-task/2214ee32-8f5b-4db0-a6c9-c5133272b749: QUEUED
⌛ Task arn:aws:braket:us-west-1:709381776175:quantum-task/2214ee32-8f5b-4db0-a6c9-c5133272b749 ainda em execução...
🔄 Status da Task arn:aws:braket:us-west-1:709381776175:quantum-task/2214ee32-8f5b-4db0-a6c9-c5133272b749: QUEUED
⌛ Task arn:aws:braket:us-west-1:709381776175:quantum-task/2214ee32-8f5b-4db0-a

KeyboardInterrupt: 